# Machine Learning to Identify Forams species and and whether the shell is cracked or not

## Finnegan Lab
## Data preprocessing

### Setup

In [1]:
# Imports. Feel free to add or remove as necessary 

!pip install keras==2.9.0rc1
!pip install -q tensorflow_addons
!pip install keras-cv-attention-models -q 

import pandas as pd
import numpy as np
np.random.seed(2022)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os, re, math
import keras
import shutil
from pathlib import Path
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds

%load_ext autoreload
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires keras<2.12,>=2.11.0, but you have keras 2.9.0rc1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.3/617.3 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.7 MB/s eta 0:00:00


In [2]:
#@title mount cloud
from google.colab import auth
auth.authenticate_user()
project_id = 'lucid-loader-347021'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://paleo-ml/


In [3]:
from google.colab import drive
drive.mount('/content/drive')
dir_drive_code ='/content/drive/MyDrive/MV1012_SBB_images/ML_projects/Code'
assert os.path.isdir(dir_drive_code), 'unable to find code. Please check your drive path above.'

Mounted at /content/drive


In [4]:
!git clone https://github.com/BryanZWu/finnegan-paleo-ml.git

Cloning into 'finnegan-paleo-ml'...
remote: Enumerating objects: 324, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 324 (delta 32), reused 54 (delta 25), pack-reused 256
Receiving objects: 100% (324/324), 110.92 KiB | 4.44 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [5]:
# @title Data stored in drive
dir_base = '/content/drive/MyDrive/MV1012_SBB_images/'
dir_code = '/content/finnegan-paleo-ml/Code'
dir_git_home = '/content/finnegan-paleo-ml'
dir_raw_training_images = f'{dir_base}/Box_Core_images'
dir_ml_home = f'{dir_base}/ML_projects'
dir_google_cloud = 'gs://paleo-ml'
dir_processed_data = dir_out_data = dir_dataset_specs = f'{dir_ml_home}/Processed_data/forams'
dir_training_logs = f'{dir_google_cloud}/Training_logs/'
dir_trained_models = f'{dir_ml_home}/Trained_models/'
dir_train_data = f'{dir_processed_data}/train'
dir_val_data = f'{dir_processed_data}/val'
dir_test_data = f'{dir_processed_data}/test'
dir_debug_processed_data = '/content/training_data_debug'
dir_labels_csv = f'{dir_base}/Final_Diversity_CSVs/ML_master2.csv'

In [6]:
os.chdir(dir_code)
#@title import local project files
from common.constants import *
from common.imports import *
from data_processing import data_processor
os.chdir(dir_git_home)

In [7]:
def git():
    """ Refetch all files """
    !git fetch
    !git checkout origin -- .

## Processing data and storing

Take data from its original form and crop/resize images as necessary. Store images with its corresponding label.

### Run processing

In [8]:
labels_df = data_processor.create_labels_df(dir_labels_csv)
labels_df.head()


Broken            species     age   test    val
sample_name  object_num                                                   
MV1012-BC-12 1.0         unbroken          NOT FORAM  1994.0  False  False
             2.0         unbroken   suggrunda eckisi  1994.0  False  False
             3.0           broken    bulimina exilis  1994.0  False  False
             4.0         unbroken  nonionella stella  1994.0  False  False
             5.0         unbroken          NOT FORAM  1994.0  False   True

In [17]:
git()

remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 9 (delta 5), reused 9 (delta 5), pack-reused 0
Unpacking objects: 100% (9/9), 3.03 KiB | 1.01 MiB/s, done.
From https://github.com/BryanZWu/finnegan-paleo-ml
 + 42c5e4f...5a570ed main       -> origin/main  (forced update)


In [19]:
dir_processed_data

'/content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams'

In [20]:
import shutil
for split in ['train', 'test', 'val']:
    shutil.rmtree(f'{dir_ml_home}/Processed_data/' + f'/{split}')



In [22]:
dir_raw_training_images

'/content/drive/MyDrive/MV1012_SBB_images//Box_Core_images'

In [26]:
#Run data processing
data_processor.run_processing(dir_processed_data, dir_raw_training_images, labels_df, verbose=True, override=True)

/content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams/train/MV1012-BC-2_obj00007.jpg
/content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams/train/MV1012-BC-2_obj00007.jpg
created $MV1012-BC-2_obj00007 in nonionella stella as train
/content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams/val/MV1012-BC-2_obj00008.jpg
/content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams/val/MV1012-BC-2_obj00008.jpg


FileNotFoundError: ignored

## Create tensorflow dataset and push to google cloud

In [ ]:
git()

In [ ]:
data_processor.create_cloud_dataset(dir_debug_processed_data, dir_google_cloud, dir_dataset_specs, 256)

<PrefetchDataset element_spec={'chamber_broken': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image': TensorSpec(shape=(416, 416, 3), dtype=tf.uint8, name=None), 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}>
